In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import optuna
import os
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:
def create_model(trial, num_classes):
    n_conv_layers = trial.suggest_int('n_conv_layers', 2, 4)

    initial_channels = trial.suggest_categorical('initial_channels', [32, 64, 128])

    conv_layers = []
    in_channels = 3
    out_channels = initial_channels

    for i in range(n_conv_layers):
        conv_layers.extend([
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        ])
        in_channels = out_channels
        out_channels = trial.suggest_categorical(f'conv_{i}_out_channels', [64, 128, 256])

    def calculate_conv_output_size():
        x = torch.randn(1, 3, 128, 128)
        conv_seq = nn.Sequential(*conv_layers)
        return conv_seq(x).view(1, -1).size(1)

    fc_input_size = calculate_conv_output_size()

    n_fc_layers = trial.suggest_int('n_fc_layers', 1, 3)
    fc_layers = []

    prev_size = fc_input_size
    for i in range(n_fc_layers):
        fc_size = trial.suggest_categorical(f'fc_{i}_size', [128, 256, 512, 1024])
        fc_layers.extend([
            nn.Linear(prev_size, fc_size),
            nn.ReLU(),
            nn.Dropout(trial.suggest_float(f'dropout_{i}', 0.2, 0.5))
        ])
        prev_size = fc_size

    fc_layers.append(nn.Linear(prev_size, num_classes))

    class OptimizedCNN(nn.Module):
        def __init__(self):
            super(OptimizedCNN, self).__init__()
            self.conv_layers = nn.Sequential(*conv_layers)
            self.fc_layers = nn.Sequential(*fc_layers)

        def forward(self, x):
            x = self.conv_layers(x)
            x = x.view(x.size(0), -1)
            x = self.fc_layers(x)
            return x

    return OptimizedCNN()

In [ ]:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    path_DS = 'animals10/raw-img'
    dataset = datasets.ImageFolder(root=path_DS, transform=transform)
    num_classes = len(dataset.classes)

    total_size = len(dataset)
    train_size = int(0.8 * total_size)
    val_size = total_size - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = create_model(trial, num_classes).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    best_val_accuracy = 0
    for epoch in range(10):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        best_val_accuracy = max(best_val_accuracy, val_accuracy)

        trial.report(val_accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_accuracy

In [ ]:
def run_hyperparameter_optimization():
    study = optuna.create_study(
        direction='maximize',
        pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=3)
    )

    study.optimize(objective, n_trials=50)

    print("Найкращі гіперпараметри:", study.best_params)
    print("Найкраща точність:", study.best_value)

    return study.best_params, study.best_value

In [ ]:
def objective(trial):
    print(f"\nПочаток тріалу {trial.number}")

    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    path_DS = 'animals10/raw-img'
    dataset = datasets.ImageFolder(root=path_DS, transform=transform)
    num_classes = len(dataset.classes)

    total_size = len(dataset)
    train_size = int(0.8 * total_size)
    val_size = total_size - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = create_model(trial, num_classes).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    epoch_accuracies = []
    for epoch in tqdm(range(10), desc=f"Навчання (LR: {lr}, Batch: {batch_size})"):
        # Навчання
        model.train()
        total_train_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        epoch_accuracies.append(val_accuracy)

        trial.report(val_accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return max(epoch_accuracies)


In [ ]:
def run_hyperparameter_optimization():
    study = optuna.create_study(
        direction='maximize',
        pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=3),
        study_name='Animal Classification Hyperparameter Optimization'
    )

    optuna.logging.enable_default_handler()
    optuna.logging.enable_propagation()
    optuna.logging.set_verbosity(optuna.logging.INFO)

    study.optimize(objective, n_trials=10)

    try:
        plt.figure(figsize=(10, 6))
        plt.title('Еволюція найкращої точності')
        plt.plot(study.trials_dataframe()['value'])
        plt.xlabel('Номер тріалу')
        plt.ylabel('Точність')
        plt.tight_layout()
        plt.savefig('hyperparameter_optimization_history.png')
        plt.close()
    except Exception as e:
        print(f"Помилка при створенні графіку: {e}")

    print("\n--- Результати оптимізації ---")
    print("Найкращі гіперпараметри:", study.best_params)
    print("Найкраща точність:", study.best_value)

    try:
        with open('hyperparameter_optimization_report.txt', 'w') as f:
            f.write("Звіт оптимізації гіперпараметрів\n")
            f.write("=====================================\n")
            f.write(f"Найкращі гіперпараметри: {study.best_params}\n")
            f.write(f"Найкраща точність: {study.best_value}\n")
            f.write("\nДеталі всіх тріалів:\n")
            for trial in study.trials:
                f.write(f"Тріал {trial.number}: Параметри {trial.params}, Точність {trial.value}\n")
    except Exception as e:
        print(f"Помилка при створенні звіту: {e}")

    return study.best_params, study.best_value

Використання Optuna для пошуку гіперпараметрів

In [30]:
best_params, best_accuracy = run_hyperparameter_optimization()

[I 2025-03-26 16:28:11,543] A new study created in memory with name: Animal Classification Hyperparameter Optimization
INFO:optuna.storages._in_memory:A new study created in memory with name: Animal Classification Hyperparameter Optimization
<ipython-input-29-cfc7458a46bf>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)



Початок тріалу 0


Навчання (LR: 0.0007380687318695055, Batch: 64): 100%|██████████| 10/10 [15:01<00:00, 90.12s/it]
[I 2025-03-26 16:43:12,870] Trial 0 finished with value: 0.6617647058823529 and parameters: {'lr': 0.0007380687318695055, 'batch_size': 64, 'n_conv_layers': 4, 'initial_channels': 128, 'conv_0_out_channels': 64, 'conv_1_out_channels': 256, 'conv_2_out_channels': 256, 'conv_3_out_channels': 64, 'n_fc_layers': 1, 'fc_0_size': 256, 'dropout_0': 0.21577401776872118}. Best is trial 0 with value: 0.6617647058823529.
INFO:optuna.study.study:Trial 0 finished with value: 0.6617647058823529 and parameters: {'lr': 0.0007380687318695055, 'batch_size': 64, 'n_conv_layers': 4, 'initial_channels': 128, 'conv_0_out_channels': 64, 'conv_1_out_channels': 256, 'conv_2_out_channels': 256, 'conv_3_out_channels': 64, 'n_fc_layers': 1, 'fc_0_size': 256, 'dropout_0': 0.21577401776872118}. Best is trial 0 with value: 0.6617647058823529.



Початок тріалу 1


Навчання (LR: 0.00010370098997556227, Batch: 64): 100%|██████████| 10/10 [15:00<00:00, 90.03s/it]
[I 2025-03-26 16:58:13,254] Trial 1 finished with value: 0.6873567608861727 and parameters: {'lr': 0.00010370098997556227, 'batch_size': 64, 'n_conv_layers': 4, 'initial_channels': 64, 'conv_0_out_channels': 256, 'conv_1_out_channels': 128, 'conv_2_out_channels': 64, 'conv_3_out_channels': 64, 'n_fc_layers': 2, 'fc_0_size': 512, 'dropout_0': 0.24734744921424534, 'fc_1_size': 1024, 'dropout_1': 0.24715336762524887}. Best is trial 1 with value: 0.6873567608861727.
INFO:optuna.study.study:Trial 1 finished with value: 0.6873567608861727 and parameters: {'lr': 0.00010370098997556227, 'batch_size': 64, 'n_conv_layers': 4, 'initial_channels': 64, 'conv_0_out_channels': 256, 'conv_1_out_channels': 128, 'conv_2_out_channels': 64, 'conv_3_out_channels': 64, 'n_fc_layers': 2, 'fc_0_size': 512, 'dropout_0': 0.24734744921424534, 'fc_1_size': 1024, 'dropout_1': 0.24715336762524887}. Best is trial 1 with

...


Початок тріалу 8


Навчання (LR: 0.002162924982376065, Batch: 64):  30%|███       | 3/10 [06:05<14:12, 121.79s/it]
[I 2025-03-26 18:08:45,087] Trial 8 pruned. 
INFO:optuna.study._optimize:Trial 8 pruned. 



Початок тріалу 9


Навчання (LR: 0.0006177692135814439, Batch: 16): 100%|██████████| 10/10 [14:16<00:00, 85.60s/it]
[I 2025-03-26 18:23:01,301] Trial 9 finished with value: 0.7337662337662337 and parameters: {'lr': 0.0006177692135814439, 'batch_size': 16, 'n_conv_layers': 4, 'initial_channels': 64, 'conv_0_out_channels': 128, 'conv_1_out_channels': 128, 'conv_2_out_channels': 128, 'conv_3_out_channels': 256, 'n_fc_layers': 2, 'fc_0_size': 128, 'dropout_0': 0.21993023251827432, 'fc_1_size': 1024, 'dropout_1': 0.411053854481769}. Best is trial 2 with value: 0.7526737967914439.
INFO:optuna.study.study:Trial 9 finished with value: 0.7337662337662337 and parameters: {'lr': 0.0006177692135814439, 'batch_size': 16, 'n_conv_layers': 4, 'initial_channels': 64, 'conv_0_out_channels': 128, 'conv_1_out_channels': 128, 'conv_2_out_channels': 128, 'conv_3_out_channels': 256, 'n_fc_layers': 2, 'fc_0_size': 128, 'dropout_0': 0.21993023251827432, 'fc_1_size': 1024, 'dropout_1': 0.411053854481769}. Best is trial 2 with va


--- Результати оптимізації ---
Найкращі гіперпараметри: {'lr': 0.00032159231595668935, 'batch_size': 64, 'n_conv_layers': 4, 'initial_channels': 64, 'conv_0_out_channels': 128, 'conv_1_out_channels': 64, 'conv_2_out_channels': 256, 'conv_3_out_channels': 128, 'n_fc_layers': 1, 'fc_0_size': 1024, 'dropout_0': 0.37519808299731244}
Найкраща точність: 0.7526737967914439
